## Variaciones sobre el modelo inicial 

Vamos a implementar alguna modificación sobre el modelo inicial propuesto en la primera sesión.

Para empezar vamos a definir un conjunto de validación. Con este conjunto de calidación extraído del propio conjunto de entrenamiento estimaremos los mejores parámetros. Dejaremos el conjunto de test sólo para estimar el acierto final con estos mejores parámetros.

Primero leemos los datos y normalizamos:

In [2]:
from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

training set (60000, 28, 28)
test set (10000, 28, 28)


## Partición training/validación

Nos quedaremos con un 80% de los datos para entrenar y un 20% para validar:

In [3]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print('training set', x_train.shape)
print('val set', x_val.shape)

training set (48000, 784)
val set (12000, 784)


## Experimentación con conjunto de validación

Cualquier parámetro a modificar en nuestro modelo debería ser probado sobre el conjunto de validación y escoger aquel que produjera el mejor resultado para probar ese (y sólo ese) sobre el conjunto de test.

In [6]:
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD
import numpy as np

batch_size=128
epochs=5  

hdim=[128,256,512,1024]
best_acc=0.0
for h in hdim:
    model = Sequential()

    model.add(Input(784))
    model.add(Dense(h, activation='relu')) # <-- repetir esta línea tantas veces como número de capas ocultas se quiera
    model.add(Dense(num_classes, activation='softmax'))

    sgd=SGD(learning_rate=0.01, momentum=0.9)


    model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
    
 

    history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val)) ## <--- OJO validation set
    
    if history.history['val_accuracy'][-1]>best_acc:
        best_acc=history.history['val_accuracy'][-1]
        besth=h

print("=============================")
print("Best acc",best_acc)
print("Best hidden dim",besth)
print("=============================")


## PROBAR EL MEJOR CON TEST
model = Sequential()

model.add(Input(784))
model.add(Dense(besth, activation='relu'))  
model.add(Dense(num_classes, activation='softmax'))

sgd=SGD(learning_rate=0.01, momentum=0.9)


model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

x_combined = np.concatenate((x_train, x_val), axis=0)
y_combined = np.concatenate((y_train, y_val), axis=0)

history = model.fit(x_combined, y_combined, # <--- aquí podríamos concatenar trainin+valid para no malgastar datos
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))  




Epoch 1/5
375/375 [==============================] - 7s 13ms/step - loss: 0.5391 - accuracy: 0.8531 - val_loss: 0.3106 - val_accuracy: 0.9143
Epoch 2/5
375/375 [==============================] - 4s 12ms/step - loss: 0.2807 - accuracy: 0.9206 - val_loss: 0.2497 - val_accuracy: 0.9314
Epoch 3/5
375/375 [==============================] - 4s 11ms/step - loss: 0.2304 - accuracy: 0.9353 - val_loss: 0.2145 - val_accuracy: 0.9417
Epoch 4/5
375/375 [==============================] - 3s 8ms/step - loss: 0.1987 - accuracy: 0.9448 - val_loss: 0.1913 - val_accuracy: 0.9470
Epoch 5/5
375/375 [==============================] - 3s 8ms/step - loss: 0.1766 - accuracy: 0.9500 - val_loss: 0.1751 - val_accuracy: 0.9516
Epoch 1/5
375/375 [==============================] - 6s 13ms/step - loss: 0.5172 - accuracy: 0.8580 - val_loss: 0.3040 - val_accuracy: 0.9155
Epoch 2/5
375/375 [==============================] - 5s 12ms/step - loss: 0.2741 - accuracy: 0.9231 - val_loss: 0.2431 - val_accuracy: 0.9320
Epoch 3/

sin train+val el resultado de acc es menor, 0.95+-


## EARLY STOPPING

De la multitud de parámetros a decidir en la definición y entrenamiento de las redes neuronales hay uno que podríamos automatizar. Es el número de epochs a emplear. Dado que tenemos un conjunto de validación vamos a emplearlo para monitorizar cómo evoluciona la convergencia. Si en algún momento el descenso por gradiente parece haber alcanzado una zona de meseta sobre dicho conjunto de validación podremos detener el entrenamiento.

In [4]:
model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

sgd=SGD(learning_rate=0.01, momentum=0.9)

callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3) ## También podría ser 'loss' sin necesitar un conjunto de validación

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

epochs=100  ## No nos preocupemos, el fit acabará antes por el early_stopping
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[callback])  



Epoch 1/100
375/375 [==============================] - 12s 26ms/step - loss: 0.4944 - accuracy: 0.8653 - val_loss: 0.2986 - val_accuracy: 0.9173
Epoch 2/100
375/375 [==============================] - 9s 25ms/step - loss: 0.2616 - accuracy: 0.9266 - val_loss: 0.2341 - val_accuracy: 0.9350
Epoch 3/100
375/375 [==============================] - 7s 19ms/step - loss: 0.2131 - accuracy: 0.9405 - val_loss: 0.1969 - val_accuracy: 0.9467
Epoch 4/100
375/375 [==============================] - 7s 20ms/step - loss: 0.1796 - accuracy: 0.9502 - val_loss: 0.1745 - val_accuracy: 0.9520
Epoch 5/100
375/375 [==============================] - 8s 21ms/step - loss: 0.1548 - accuracy: 0.9572 - val_loss: 0.1572 - val_accuracy: 0.9559
Epoch 6/100
375/375 [==============================] - 8s 21ms/step - loss: 0.1361 - accuracy: 0.9624 - val_loss: 0.1443 - val_accuracy: 0.9597
Epoch 7/100
375/375 [==============================] - 8s 20ms/step - loss: 0.1212 - accuracy: 0.9670 - val_loss: 0.1332 - val_accuracy

## **Ejercicio**:

Entrenar una red neuronal MLP para el problema MNIST. Los valores que se quieren probar son los siguientes:

-   Número de capas ocultas [1,2,3]
-   Dimensión de las capas ocultas [512,1024]
-   Learning rate [0.025, 0.01]
-   Batch_size [128]
-   Epochs con early_stopping

Obtener la mejor combinación con el conjunto de validación y probarla finalmente sobre el conjunto de test.

---

In [10]:
num_capas_ocultas = [1,2,3]
dim_capas_ocultas = [512, 1024]
learningRate = [0.025, 0.01]
batch_size = 128
param = []
for lr in learningRate:
    for n in num_capas_ocultas:
        for d in dim_capas_ocultas:
            model = Sequential()
            model.add(Input(784))
            for numCapas in range(n):
                model.add(Dense(d, activation='relu'))
            model.add(Dense(num_classes, activation='softmax'))

            sgd=SGD(learning_rate=lr, momentum=0.9)

            callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3) ## También podría ser 'loss' sin necesitar un conjunto de validación

            model.compile(loss='categorical_crossentropy',
                        optimizer=sgd,
                        metrics=['accuracy'])

            epochs=100  ## No nos preocupemos, el fit acabará antes por el early_stopping
            history = model.fit(x_train, y_train,
                                batch_size=batch_size,
                                epochs=epochs,
                                verbose=1,
                                validation_data=(x_val, y_val),
                                callbacks=[callback])  
            if history.history['val_accuracy'][-1]>best_acc:
                best_acc=history.history['val_accuracy'][-1]
                besth=h
                param = "Learning rate: " + str(lr) +" \n Numero de capas ocultas: " + str(n) + "Dimension capas ocultas: " +str(d)


print("=============================")
print("Best acc",best_acc)
print("Best hidden dim",besth)
print("..............Best param........ \n" , param)
print("=============================")

## PROBAR EL MEJOR CON TEST
model = Sequential()
model.add(Input(784))
model.add(Dense(besth, activation='relu'))  
model.add(Dense(num_classes, activation='softmax'))
sgd=SGD(learning_rate=0.01, momentum=0.9)
callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3) ## También podría ser 'loss' sin necesitar un conjunto de validación
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
x_combined = np.concatenate((x_train, x_val), axis=0)
y_combined = np.concatenate((y_train, y_val), axis=0)
history = model.fit(x_combined, y_combined, # <--- aquí podríamos concatenar trainin+valid para no malgastar datos
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[callback]) 

Epoch 1/100
375/375 [==============================] - 9s 20ms/step - loss: 0.3816 - accuracy: 0.8907 - val_loss: 0.2211 - val_accuracy: 0.9374
Epoch 2/100
375/375 [==============================] - 7s 19ms/step - loss: 0.1865 - accuracy: 0.9468 - val_loss: 0.1626 - val_accuracy: 0.9533
Epoch 3/100
375/375 [==============================] - 7s 18ms/step - loss: 0.1378 - accuracy: 0.9611 - val_loss: 0.1331 - val_accuracy: 0.9616
Epoch 4/100
375/375 [==============================] - 7s 19ms/step - loss: 0.1091 - accuracy: 0.9695 - val_loss: 0.1146 - val_accuracy: 0.9668
Epoch 5/100
375/375 [==============================] - 6s 16ms/step - loss: 0.0890 - accuracy: 0.9754 - val_loss: 0.1009 - val_accuracy: 0.9702
Epoch 6/100
375/375 [==============================] - 6s 17ms/step - loss: 0.0745 - accuracy: 0.9795 - val_loss: 0.0944 - val_accuracy: 0.9722
Epoch 7/100
375/375 [==============================] - 6s 17ms/step - loss: 0.0640 - accuracy: 0.9828 - val_loss: 0.0876 - val_accuracy:

KeyboardInterrupt: 

## Model checkpoint

Los model checkpoint permiten almacenar el estado del modelo en un punto intermedio del entrenamiento. Esto es útil para poder recuperar el modelo en caso de que el entrenamiento se interrumpa por cualquier motivo. Así como para poder recuperar el mejor modelo obtenido durante el entrenamiento.


In [5]:
batch_size=128
epochs=100 

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu')) 
model.add(Dense(num_classes, activation='softmax'))

sgd=SGD(learning_rate=0.01, momentum=0.9)


model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
    
 

callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3) 

checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[callback, checkpoint]) ## <--- dos callbacks, el early_stopping y el model_checkpoint


## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])





Epoch 1/100
370/375 [============================>.] - ETA: 0s - loss: 0.4950 - accuracy: 0.8666
Epoch 1: val_accuracy improved from -inf to 0.91442, saving model to best_model.h5
375/375 [==============================] - 4s 10ms/step - loss: 0.4922 - accuracy: 0.8673 - val_loss: 0.2980 - val_accuracy: 0.9144
Epoch 2/100
 14/375 [>.............................] - ETA: 3s - loss: 0.2746 - accuracy: 0.9269

c:\Users\david\anaconda3\envs\vsc\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


375/375 [==============================] - ETA: 0s - loss: 0.2613 - accuracy: 0.9265
Epoch 2: val_accuracy improved from 0.91442 to 0.93392, saving model to best_model.h5
375/375 [==============================] - 4s 10ms/step - loss: 0.2613 - accuracy: 0.9265 - val_loss: 0.2340 - val_accuracy: 0.9339
Epoch 3/100
374/375 [============================>.] - ETA: 0s - loss: 0.2114 - accuracy: 0.9412
Epoch 3: val_accuracy improved from 0.93392 to 0.94600, saving model to best_model.h5
375/375 [==============================] - 4s 10ms/step - loss: 0.2113 - accuracy: 0.9412 - val_loss: 0.1972 - val_accuracy: 0.9460
Epoch 4/100
370/375 [============================>.] - ETA: 0s - loss: 0.1781 - accuracy: 0.9507
Epoch 4: val_accuracy improved from 0.94600 to 0.95225, saving model to best_model.h5
375/375 [==============================] - 4s 11ms/step - loss: 0.1782 - accuracy: 0.9505 - val_loss: 0.1741 - val_accuracy: 0.9523
Epoch 5/100
375/375 [==============================] - ETA: 0s - lo

## Ajustar parámetros

Es posible que en el ejemplo anterior el entrenamiento haya acabado prematuramente por el efecto del Early Stopping. En ese caso, el modelo que se ha obtenido no es el mejor posible. Para obtener el mejor modelo quizás sea necesario aumentar el número de epochs y para ello modificar algún parámetro del early stopping.

## Ejercicio: 

Modificar el early stopping y emplear un modelo con los mejores parámetros (número de capas ocultas, learning rate, batch size, etc) para evaluar finalmente el modelo guardado sobre el test.


In [ ]:
batch_size=128
epochs=100 

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu')) 
model.add(Dense(num_classes, activation='softmax'))

sgd=SGD(learning_rate=0.01, momentum=0.9)


model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
    
 

callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3) 

checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[callback, checkpoint]) ## <--- dos callbacks, el early_stopping y el model_checkpoint


## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])